In [ ]:
!ldconfig

In [ ]:
%pip install git+ssh://git@gitlab-master.nvidia.com:12051/earth-2/fcn-mip.git@main

In [ ]:
from fcn_mip.initial_conditions.cds import get
from fcn_mip.schema import  ChannelSet
import os
from fcn_mip import schema, weather_events
from fcn_mip.initial_conditions import get
import datetime

time = datetime.datetime(2021, 9, 10, 0)

kw = dict(time=time, channel_set=schema.ChannelSet.var73, n_history=0)
ic= get(source=weather_events.InitialConditionSource.cds, **kw)
# need to hack around bug in cds source
# https://gitlab-master.nvidia.com/earth-2/fcn-mip/-/issues/56
cds = ic.roll(lon=720)
cds.rename("fields").to_netcdf("ic.nc")

In [ ]:
%%file config.json
{
    "simulation_length": 12,
    "weather_event": {
      "properties": {
        "name": "Globe",
        "netcdf": "ic.nc"
      },
      "domains": [
        {
          "name": "global",
          "type": "Window",
          "diagnostics": [
            {
              "type": "raw",
              "channels": [
                  "tcwv",
                  "z500",
                  "t500",
                  "u500",
                  "v500",
                  "z700",
                  "t700",
                  "u700",
                  "v700",
                  "z850",
                  "t850",
                  "u850",
                  "v850",
                  "z925",
                  "t925",
                  "u925",
                  "v925",
                  "t2m",
                  "u10m",
                  "v10m"
                ]

            }
          ]
        }
      ]
    },
    "output_path": "output",
    "output_frequency": 1,
    "fcn_model": "sfno_coszen"
}

In [ ]:
%%bash
export WORLD_SIZE=1
export LOCAL_CACHE=/workspace/.cache
rm -rf output/
python3 -m fcn_mip.inference_ensemble config.json

In [2]:
import xarray

root = xarray.open_dataset("output/ensemble_out_0.nc")
output = xarray.open_dataset("output/ensemble_out_0.nc", group='global').merge(root)
output

/usr/local/lib/python3.8/dist-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.21.0 or higher is recommended. You are running version 2.16.0
  warnings.warn(


<xarray.Dataset>
Dimensions:  (lat: 721, lon: 1440, ensemble: 1, time: 13)
Coordinates:
  * lat      (lat) float32 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * lon      (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
  * time     (time) datetime64[ns] 2021-09-10 2021-09-10T06:00:00 ... 2021-09-13
Dimensions without coordinates: ensemble
Data variables: (12/24)
    imin     int64 ...
    imax     int64 ...
    jmin     int64 ...
    jmax     int64 ...
    tcwv     (ensemble, time, lat, lon) float64 ...
    z500     (ensemble, time, lat, lon) float64 ...
    ...       ...
    t925     (ensemble, time, lat, lon) float64 ...
    u925     (ensemble, time, lat, lon) float64 ...
    v925     (ensemble, time, lat, lon) float64 ...
    t2m      (ensemble, time, lat, lon) float64 ...
    u10m     (ensemble, time, lat, lon) float64 ...
    v10m     (ensemble, time, lat, lon) float64 ...

In [8]:
channels = [
                  "tcwv",
                  "z500",
                  "t500",
                  "u500",
                  "v500",
                  "z700",
                  "t700",
                  "u700",
                  "v700",
                  "z850",
                  "t850",
                  "u850",
                  "v850",
                  "z925",
                  "t925",
                  "u925",
                  "v925",
                  "t2m",
                  "u10m",
                  "v10m"
                ]

cwb_path = "/lustre/fsw/sw_climate_fno/nbrenowitz/2023-01-24-cwb-4years.zarr"
cwb = xarray.open_zarr(cwb_path)
xlat = cwb["XLAT"]
xlong = cwb["XLONG"]
p = output.to_array(dim='channel', name='fields').sel(channel=channels).isel(ensemble=0)
interpolated = p.interp(lat=xlat, lon=xlong)
interpolated = interpolated.transpose("time", "channel", "south_north", "west_east")

dataset = interpolated.to_dataset()
dataset['target'] = cwb.cwb.sel(time=output.time)

dataset.to_netcdf("fcn_outputs.nc")

In [10]:
%%bash

cd ../../../
./test.sh

/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: File /lib/x86_64-linux-gnu/libnvidia-ml.so.510.47.03 is empty, not checked.
/sbin/ldconfig.real: File /lib/x86_64-linux-gnu/libnvidia-encode.so.510.47.03 is empty, not checked.
/sbin/ldconfig.real: File /lib/x86_64-linux-gnu/libnvidia-ptxjitcompiler.so.510.47.03 is empty, not checked.
/sbin/ldconfig.real: File /lib/x86_64-linux-gnu/libnvidia-allocator.so.510.47.03 is empty, not checked.
/sbin/ldconfig.real: File /lib/x86_64-linux-gnu/libnvidia-opencl.so.510.47.03 is empty, not checked.
/sbin/ldconfig.re

...

generations/netcdf/fcn


++ ncfile=generations/netcdf/fcn/samples.nc
++ [[ -f generations/netcdf/fcn/samples.nc ]]
++ directory=generations/netcdf/fcn/multiple
++ mkdir -p generations/netcdf/fcn/multiple
++ python3 plot_multiple_samples.py generations/netcdf/fcn/samples.nc generations/netcdf/fcn/multiple
/usr/local/lib/python3.8/dist-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.21.0 or higher is recommended. You are running version 2.16.0
  warnings.warn(


maximum_radar_reflectivity
eastward_wind_10m
temperature_2m
northward_wind_10m


Traceback (most recent call last):
  File "plot_multiple_samples.py", line 37, in <module>
    main()
  File "/root/.local/lib/python3.8/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/root/.local/lib/python3.8/site-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/root/.local/lib/python3.8/site-packages/click/core.py", line 1404, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/root/.local/lib/python3.8/site-packages/click/core.py", line 760, in invoke
    return __callback(*args, **kwargs)
  File "plot_multiple_samples.py", line 35, in main
    joblib.Parallel(n_jobs=8)(joblib.delayed(plot)(v) for v in merged)
  File "/usr/local/lib/python3.8/dist-packages/joblib/parallel.py", line 1098, in __call__
    self.retrieve()
  File "/usr/local/lib/python3.8/dist-packages/joblib/parallel.py", line 975, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  File "/usr/local/lib

CalledProcessError: Command 'b'\ncd ../../../\n./test.sh\n'' returned non-zero exit status 1.

In [6]:
!apt-get update && apt-get install -y imagemagick

Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:1 https://packagecloud.io/github/git-lfs/ubuntu focal InRelease [24.4 kB]  
Get:4 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:3 https://packagecloud.io/github/git-lfs/ubuntu focal/main amd64 Packages [3134 B]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2854 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2479 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]      
Get:9 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [28.5 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1066 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Ge

In [7]:
%%bash

cd ../../../

convert -delay 25 -loop 0 generations/netcdf/fcn/singlke/*.sample.png generations/netcdf/sample.gif

convert-im6.q16: unable to open image `generations/netcdf/fcn/singlke/*.sample.png': No such file or directory @ error/blob.c/OpenBlob/2874.
convert-im6.q16: no images defined `generations/netcdf/sample.gif' @ error/convert.c/ConvertImageCommand/3258.


CalledProcessError: Command 'b'\ncd ../../../\n\nconvert -delay 25 -loop 0 generations/netcdf/fcn/singlke/*.sample.png generations/netcdf/sample.gif\n'' returned non-zero exit status 1.

In [ ]:
from IPython.display import display, HTML
import base64

def embed_gif(path):
    b64 = base64.b64encode(open(path,'rb').read()).decode('ascii')
    return HTML(f'<img src="data:image/gif;base64,{b64}" />')

display(embed_gif("../../../generations/netcdf/sample.gif"))